# IJO1366-Tyr-Phe-Auxotrophs

This notebook contains simulations for IJO1366-Tyr-Phe-Auxotrophs environment where two Ecoli auxotroph agents are learning to interact from scratch. For this simulation we need to use faster LP solvers 
as we are dealing with very large metabolic models. Here we used Gurobi. 

In [1]:
from spamdfba import toolkit as tk
from spamdfba import toymodels as tm
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import os
import warnings
import rich
import json
import cobra

2023-10-31 11:15:37,533	INFO worker.py:1642 -- Started a local Ray instance.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-17


In [2]:
NUM_CORES = 8
warnings.filterwarnings("ignore")

In [3]:
model_base = cobra.io.read_sbml_model("iJO1366.xml")
medium = model_base.medium.copy()
test_model = model_base.copy()

knockouts_gene_names = [
    "tyrA",
    "pheA",
]

exchange_reactions = {
    "tyrA": "EX_tyr__L_e",
    "pheA": "EX_phe__L_e",
}

exchange_species = {}
exchange_mets = {}
for i in exchange_reactions.items():
    exchange_mets[i[0]] = list(test_model.reactions.get_by_id(i[1]).metabolites.keys())[
        0
    ].id

gene_ids = {}
for ko_gene in knockouts_gene_names:
    for gene in test_model.genes:
        if gene.name == ko_gene:
            gene_ids[ko_gene] = gene.id

            




ic={
    key.lstrip("EX_"):3 for key,val in model_base.medium.items() 
}

ic['glc__D_e']=500
ic['agent1']=0.1
ic['agent2']=0.1
for ko in [("tyrA","pheA")]:
    model1 = model_base.copy()
    model2 = model_base.copy()
    model1.remove_reactions([model1.reactions.get_by_id('PPND')]) ## Tyrosine Mutant
    model2.remove_reactions([model2.reactions.get_by_id('PPNDH')]) ## Phenylalanine Mutant
    model1.exchange_reactions = tuple([model1.reactions.index(i) for i in model1.exchanges])
    model2.exchange_reactions = tuple([model2.reactions.index(i) for i in model2.exchanges])
    model1.biomass_ind=model1.reactions.index("BIOMASS_Ec_iJO1366_core_53p95M")
    model2.biomass_ind=model2.reactions.index("BIOMASS_Ec_iJO1366_core_53p95M")
    model1.solver = "gurobi"
    model2.solver = "gurobi"
    if model1.optimize().objective_value != 0 or model2.optimize().objective_value != 0:
        rich.print(f"[yellow]Skipping {ko} because at least one organism can grow without auxotrophy")
        continue
    else:
        rich.print(f"[green]Non of the KOs can grow without auxotrophy: Running {ko}")
    ko_name = ko[0] + "_" + ko[1]
    agent1 = tk.Agent(
        "agent1",
        model=model1,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        actor_var=0.05,
        grad_updates=10,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            *[i.replace("EX_", "") for i in exchange_reactions.values()]
        ],
        actions=[i for i in exchange_reactions.values()],
        gamma=1,
    )
    agent2 = tk.Agent(
        "agent2",
        model=model2,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        grad_updates=10,
        actor_var=0.05,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            *[i.replace("EX_", "") for i in exchange_reactions.values()]
        ],
        actions=[i for i in exchange_reactions.values()],
        gamma=1,
    )
    constants=list(ic.keys())
    constants.remove("agent1")
    constants.remove("agent2")
    constants.remove("glc__D_e")

    env = tk.Environment(
        "IJO1366-Tyr-Phe-Auxotrophs" ,
        agents=[agent1, agent2],
        dilution_rate=0,
        extracellular_reactions=[],
        initial_condition=ic,
        inlet_conditions={},
        dt=0.2,
        episode_length=250,
        number_of_batches=5000,  ##TOBECHANGED
        episodes_per_batch=4,
        constant=constants,
    )


Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpi5v_02_k.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpef33aako.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpq5d6ql7_.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


Non of the KOs can grow without auxotrophy: Running ('tyrA', 'pheA')

In [4]:
sim=tk.Simulation(name=env.name,
                  env=env,
                  save_dir="./Results/",
                  )

In [5]:
sim.run()

Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Batch 0 finished:
agent1 return was:  -243.49904137102834
agent2 return was:  -3.8932280865355215
Batch 1 finished:
agent1 return was:  -243.49993400732856
agent2 return was:  -2.091909251316706
Batch 2 finished:
agent1 return was:  -245.7498467138368
agent2 return was:  -1.8566647029372003
Batch 3 finished:
agent1 return was:  -242.74972482033905
agent2 return was:  -2.0573058854160005
Batch 4 finished:
agent1 return was:  -245.74967254755066
agent2 return was:  -3.616969832185714
Batch 5 finished:
agent1 return was:  -239.49962793444107
agent2 return was:  -1.4744184162897909
Batch 6 finished:
agent1 return was:  -240.99888318578172
agent2 return was:  -1.2095130484877235
Batch 7 finished:
agent1 return was:  -240.74955883278585
agent2 return was:  -1.8686368164421996
Batch 8 finished:
agent1 return was:  -242.49925705080025
agent2 return was:  -2.8461559764234163
Batch 9 f

In [6]:
sim.plot_learning_curves()

In [7]:
sim.print_training_times()

                       Simulation times                       
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Level        ┃ Mean(s)             ┃ STD(s)                ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Optimization │ 0.02075670466423035 │ 0.001700334016565152  │
│ Step         │ 0.05957710268497467 │ 0.0045810640569996595 │
│ Batch        │ 16.75212166309357   │ 1.1049193948314588    │
│ Simulation   │ 176.28628396987915  │ NA                    │
└──────────────┴─────────────────────┴───────────────────────┘

[{'mean': 0.05957710268497467, 'std': 0.0045810640569996595},
 {'mean': 0.02075670466423035, 'std': 0.001700334016565152},
 {'mean': 16.75212166309357, 'std': 1.1049193948314588},
 {'mean': 176.28628396987915, 'std': 0.0}]